## Getting Data From APIs

What is an API?
- Application Programming Interface
- Structured way to expose specific functionality and data access to users
- Web APIs usually follow the "REST" standard

How to interact with a REST API:
- Make a "request" to a specific URL (an "endpoint"), and get the data back in a "response"
- Most relevant request method for us is GET (other methods: POST, PUT, DELETE)
- Response is often JSON format
- Web console is sometimes available (allows you to explore an API)

### Read the IMDb data into a DataFrame: we want a year column!


In [1]:
import pandas as pd
movies = pd.read_csv('../../DAT-DC-10/data/imdb_1000.csv')
movies.head()

,star_rating,title,content_rating,genre,duration,actors_list
0,9.3,The Shawshank Redemption,R,Crime,142,"[u'Tim Robbins', u'Morgan Freeman', u'Bob Gunt..."
1,9.2,The Godfather,R,Crime,175,"[u'Marlon Brando', u'Al Pacino', u'James Caan']"
2,9.1,The Godfather: Part II,R,Crime,200,"[u'Al Pacino', u'Robert De Niro', u'Robert Duv..."
3,9.0,The Dark Knight,PG-13,Action,152,"[u'Christian Bale', u'Heath Ledger', u'Aaron E..."
4,8.9,Pulp Fiction,R,Crime,154,"[u'John Travolta', u'Uma Thurman', u'Samuel L...."


In [2]:
movies.dtypes

star_rating       float64
title              object
content_rating     object
genre              object
duration            int64
actors_list        object
dtype: object

We can use the library **requests** to interact with a URL and webpages.

**Homework:** Read up on the requests library

In [7]:
import requests
r = requests.get('http://www.omdbapi.com/?t=the shawshank redemption&r=json&type=movie')

In [8]:
# check the status: 200 means success, 4xx means error
r.status_code

200

In [9]:
# view the raw response text
r.text

u'{"Title":"The Shawshank Redemption","Year":"1994","Rated":"R","Released":"14 Oct 1994","Runtime":"142 min","Genre":"Crime, Drama","Director":"Frank Darabont","Writer":"Stephen King (short story \\"Rita Hayworth and Shawshank Redemption\\"), Frank Darabont (screenplay)","Actors":"Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler","Plot":"Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.","Language":"English","Country":"USA","Awards":"Nominated for 7 Oscars. Another 14 wins & 19 nominations.","Poster":"http://ia.media-imdb.com/images/M/MV5BODU4MjU4NjIwNl5BMl5BanBnXkFtZTgwMDU2MjEyMDE@._V1_SX300.jpg","Metascore":"80","imdbRating":"9.3","imdbVotes":"1,569,613","imdbID":"tt0111161","Type":"movie","Response":"True"}'

### Decode the JSON response body into a python data structure

#### Quiz: What is the Data Structure and how would we look at 'Year' that the movie came out?

In [10]:
r.json()['Plot']

u'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.'

### What happens if the movie name is not recognized by the API?

In [11]:
r = requests.get('http://www.omdbapi.com/?t=blahblahblah&r=json&type=movie')
r.status_code
r.json() #return python dictionary

{u'Error': u'Movie not found!', u'Response': u'False'}

## Let's define a function to return the year

In [12]:
def get_movie_year(title):
    r = requests.get('http://www.omdbapi.com/?t=' + title + '&r=json&type=movie')
    info = r.json()
    if info['Response'] == 'True':
        try:
            return int(info['Year'])
        except:
            return 'NA'
    else:
        return "No such movie"

In [13]:
def get_movie_year_v2(title, detail):
    detail = detail.capitalize()
    r = requests.get('http://www.omdbapi.com/?t=' + title + '&r=json&type=movie')
    info = r.json()
    if info['Response'] == 'True':
        try:
            return info[detail]
        except:
            return 'NA'
    else:
        return "No such movie"

### Let's test that the function works

In [14]:
get_movie_year('The Shawshank Redemption')

1994

In [15]:
get_movie_year('blahblahblah')

'No such movie'

## Let's do something cool with our new function

#### Create a smaller DataFrame for testing

In [16]:
top_movies = movies.head().copy()
top_movies

,star_rating,title,content_rating,genre,duration,actors_list
0,9.3,The Shawshank Redemption,R,Crime,142,"[u'Tim Robbins', u'Morgan Freeman', u'Bob Gunt..."
1,9.2,The Godfather,R,Crime,175,"[u'Marlon Brando', u'Al Pacino', u'James Caan']"
2,9.1,The Godfather: Part II,R,Crime,200,"[u'Al Pacino', u'Robert De Niro', u'Robert Duv..."
3,9.0,The Dark Knight,PG-13,Action,152,"[u'Christian Bale', u'Heath Ledger', u'Aaron E..."
4,8.9,Pulp Fiction,R,Crime,154,"[u'John Travolta', u'Uma Thurman', u'Samuel L...."


#### Write a for loop to build a list of years

In [17]:
from time import sleep
years = []
for title in top_movies.title:
    years.append(get_movie_year(title))
    sleep(3)
    print title

The Shawshank Redemption
The Godfather
The Godfather: Part II
The Dark Knight
Pulp Fiction


### Quiz: What is the sleep function and why is it a good idea to use it?

#### Check that the DataFrame and the list of years are the same length

In [96]:
assert(len(top_movies) == len(years))
years

[1994, 1972, 1974, 2008, 1994]

### Quiz: How would we save the list of years to a new column in top movies?

In [98]:
top_movies['Years'] = years
# top_movies.insert(0, 'year', years)
top_movies

,star_rating,title,content_rating,genre,duration,actors_list,Years
0,9.3,The Shawshank Redemption,R,Crime,142,"[u'Tim Robbins', u'Morgan Freeman', u'Bob Gunt...",1994
1,9.2,The Godfather,R,Crime,175,"[u'Marlon Brando', u'Al Pacino', u'James Caan']",1972
2,9.1,The Godfather: Part II,R,Crime,200,"[u'Al Pacino', u'Robert De Niro', u'Robert Duv...",1974
3,9.0,The Dark Knight,PG-13,Action,152,"[u'Christian Bale', u'Heath Ledger', u'Aaron E...",2008
4,8.9,Pulp Fiction,R,Crime,154,"[u'John Travolta', u'Uma Thurman', u'Samuel L....",1994


** Bonus Content (Time Permitting) **

In [ ]:
# enumerate allows you to access the item location while iterating
letters = ['a', 'b', 'c']
for index, letter in enumerate(letters):
    print index, letter

In [ ]:
# iterrows method for DataFrames is similar
for index, row in top_movies.iterrows():
    print index, row.title

In [ ]:
# create a new column and set a default value
movies['year'] = -1

In [ ]:
# loc method allows you to access a DataFrame element by 'label'
movies.loc[0, 'year'] = 1994

In [ ]:
# write a for loop to update the year for the first three movies
for index, row in movies.iterrows():
    if index < 3:
        movies.loc[index, 'year'] = get_movie_year(row.title)
        sleep(1)
    else:
        break

Other considerations when accessing APIs:
- Most APIs require you to have an access key (which you should store outside your code)
- Most APIs limit the number of API calls you can make (per day, hour, minute, etc.)
- Not all APIs are free
- Not all APIs are well-documented
- Pay attention to the API version
Python wrapper is another option for accessing an API:
- Set of functions that "wrap" the API code for ease of use
- Potentially simplifies your code
- But, wrapper could have bugs or be out-of-date or poorly documented